# Temporal Scope Tutorial: Health Monitoring Analysis

## Overview

This tutorial demonstrates how to analyze temporal biological data using the **TemporalScope** framework. We'll work with multiple health metrics to showcase both machine learning and deep learning approaches to temporal analysis.

### Summary

| **Step**  | **Description**                                                                 |
|-----------|---------------------------------------------------------------------------------|
| **1**     | **Data Generation**: Create synthetic health data with realistic patterns        |
| **2**     | **TimeFrame Setup**: Initialize temporal data structures for each health metric  |
| **3**     | **ML Processing**: Prepare data for one-step-ahead forecasting                  |
| **4**     | **DL Processing**: Prepare sequence data for deep learning models               |
| **5**     | **Temporal Splits**: Create proper train/test partitions                        |

### Key Concepts

- **Multiple Health Metrics**: Blood pressure, stress levels, and heart rate
- **Temporal Patterns**: Daily, weekly, and seasonal variations
- **Forecasting Approaches**: Both one-step-ahead and sequence-based predictions
- **Proper Validation**: Time-aware train/test splitting

### Steps

1. **Generate Health Data**
   - Create synthetic but realistic health measurements
   - Include known physiological patterns and correlations

2. **Initialize TimeFrames**
   - Separate temporal structures for each health metric
   - Enable parallel processing capabilities

3. **Prepare Forecasting Data**
   - Machine learning mode for immediate predictions
   - Deep learning mode for sequence-based analysis

4. **Create Temporal Splits**
   - Sliding window approach
   - Maintain temporal ordering
   - Multiple validation periods

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from temporalscope.core.temporal_data_loader import TimeFrame
from temporalscope.core.temporal_target_shifter import TemporalTargetShifter
from temporalscope.partition.sliding_window import SlidingWindowPartitioner
from temporalscope.core.core_utils import print_divider

In [ ]:
def generate_health_data(start_date: str = '2023-01-01', days: int = 365):
    """Generate synthetic health monitoring data.
    
    Args:
        start_date (str): Starting date for the data
        days (int): Number of days to generate
    """
    # Create date range for daily measurements
    dates = pd.date_range(start=start_date, periods=days, freq='D')
    
    # Time array for generating patterns
    t = np.arange(days)
    
    # Seasonal effect (yearly cycle)
    # - Amplitude of 5 represents typical seasonal BP variation
    # - 2π/365 gives us one complete cycle per year
    seasonal_effect = 5 * np.sin(2 * np.pi * t / 365)
    
    # Weekly pattern (work week stress)
    # - Amplitude of 3 for weekly BP fluctuation
    # - 2π/7 gives us one complete cycle per week
    weekly_effect = 3 * np.sin(2 * np.pi * t / 7)
    
    # Blood Pressure Generation
    # Systolic (120 typical baseline)
    # - Stronger influence from seasonal & weekly patterns
    # - Random variation (σ=3) for daily fluctuations
    systolic = 120 + seasonal_effect + weekly_effect + np.random.normal(0, 3, days)
    
    # Diastolic (80 typical baseline)
    # - Less affected by external patterns (multiplied by 0.5)
    # - Smaller random variation (σ=2)
    diastolic = 80 + seasonal_effect * 0.5 + weekly_effect * 0.5 + np.random.normal(0, 2, days)
    
    # Stress Level Generation (0-100 scale)
    # - Heavily influenced by weekly pattern (work stress)
    # - Larger random variation (σ=5) for daily life events
    # - Clipped to valid range [0,100]
    stress = 50 + weekly_effect + np.random.normal(0, 5, days)
    stress = np.clip(stress, 0, 100)
    
    # Heart Rate Generation
    # - Baseline of 70 bpm
    # - Correlates with stress (0.3 coefficient)
    # - Weekly pattern influence
    # - Moderate random variation (σ=3)
    heart_rate = 70 + 0.3 * stress + weekly_effect + np.random.normal(0, 3, days)
    
    return pd.DataFrame({
        'ds': dates,
        'systolic': systolic,
        'diastolic': diastolic,
        'stress_level': stress,
        'heart_rate': heart_rate
    })

In [ ]:
def create_metric_timeframes(df):
    """Create TimeFrame objects for each health metric.
    
    Why separate TimeFrames?
    - Each metric might need different forecasting horizons
    - Allows parallel processing of different metrics
    - Can apply different temporal transformations per metric
    """
    metrics = ['systolic', 'diastolic', 'stress_level', 'heart_rate']
    timeframes = {}
    
    for metric in metrics:
        # Using pandas backend for simplicity
        # Could switch to Modin/Polars for larger datasets
        timeframes[metric] = TimeFrame(
            df=df,
            time_col='ds',  # datetime column
            target_col=metric,  # metric to forecast
            backend='pd'
        )
    
    return timeframes

In [ ]:
def prepare_forecasting_data(timeframe, mode='machine_learning', sequence_length=7):
    """Prepare data for forecasting using TemporalTargetShifter.
    
    Two modes supported:
    1. Machine Learning (ml) mode:
       - One-step-ahead prediction
       - Useful for immediate forecasts (next day)
       - Better for interpretable models (regression, etc.)
    
    2. Deep Learning (dl) mode:
       - Sequence-to-sequence prediction
       - Captures longer temporal patterns
       - Better for complex patterns (LSTM, etc.)
       - sequence_length=7 for weekly patterns
    """
    shifter = TemporalTargetShifter(
        n_lags=1,  # How many steps to look ahead
        mode=mode,
        sequence_length=sequence_length if mode == 'deep_learning' else None,
        verbose=True
    )
    
    return shifter.fit_transform(timeframe)

In [ ]:
def create_temporal_splits(timeframe, num_partitions=3):
    """Create temporal train/test splits using sliding window.
    
    Why sliding window?
    - Maintains temporal ordering (crucial for time series)
    - Multiple partitions to assess model stability
    - Each partition moves forward in time
    - 70/30 split preserves enough history for training
    
    Why num_partitions=3?
    - Tests model on different time periods
    - Captures seasonal variations
    - Balance between validation and data usage
    """
    partitioner = SlidingWindowPartitioner(
        tf=timeframe,
        num_partitions=num_partitions,  # Number of temporal splits
        train_pct=0.7,  # 70% for training
        test_pct=0.3    # 30% for testing
    )
    
    return list(partitioner.fit_transform())

In [ ]:
if __name__ == "__main__":
    # Step 1: Generate synthetic health data
    print_divider()
    print("Generating synthetic health data...")
    health_df = generate_health_data()
    print("Preview of generated health data:")
    print(health_df.head())
    print_divider()
    
    # Step 2: Create TimeFrames for each metric
    print("Initializing TimeFrames for each health metric...")
    metric_timeframes = create_metric_timeframes(health_df)
    
    # Step 3: Demonstrate both ML and DL approaches
    print("\nPreparing data for different forecasting approaches:")
    for metric in ['heart_rate', 'stress_level']:
        print(f"\nProcessing {metric}:")
        
        # ML mode (one-step-ahead)
        print("\nMachine Learning mode (one-step-ahead):")
        ml_data = prepare_forecasting_data(metric_timeframes[metric], mode='machine_learning')
        print(ml_data.head())
        
        # DL mode (sequence)
        print("\nDeep Learning mode (sequence-based):")
        dl_data = prepare_forecasting_data(metric_timeframes[metric], mode='deep_learning')
        print(dl_data.head())
        
        print_divider()
    
    # Step 4: Create and demonstrate temporal splits
    print("\nCreating temporal splits for validation:")
    heart_rate_splits = create_temporal_splits(metric_timeframes['heart_rate'])
    
    for i, partition in enumerate(heart_rate_splits):
        print(f"\nPartition {i+1}:")
        print(f"Train shape: {partition['partition_1']['train'].shape}")
        print(f"Test shape: {partition['partition_1']['test'].shape}")
    
    print_divider()